#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [7]:
import pandas as pd
import os
data_path = '../../data/supermarket_sales_ML.csv'
df = pd.read_csv(data_path)

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [11]:
df.groupby('Branch')['gross income'].agg('sum')

Branch
A    5057.1605
B    5057.0320
C    5265.1765
Name: gross income, dtype: float64

2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [31]:
df.groupby('Branch')['City'].describe()

,count,unique,top,freq
Branch,,,,
A,340,1,Yangon,340
B,332,1,Mandalay,332
C,328,1,Naypyitaw,328


# Only one branch is shown in each city.
Branch 'C' with the highest gross income (5265.1765) is located in the city 'Naypyitaw'.

3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [148]:
invoices_by_category = pd.pivot_table(df, index='Branch', columns='Product line', aggfunc='size')

In [150]:
invoices_by_category

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [142]:
condition_1 = invoices_by_category['Electronic accessories'] == invoices_by_category['Electronic accessories'].max()

In [144]:
invoices_by_category[condition_1]['Electronic accessories']

Branch
A    60
Name: Electronic accessories, dtype: int64

4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


4-6. **Гіпотеза 1**

In [511]:
invoices_by_customer_type_rating = pd.pivot_table(df, index=('Branch','Product line'), columns='Customer type', values= 'Rating' ,aggfunc='mean').round(1).reset_index()

In [543]:
invoices_by_customer_type_rating['Member Normal rating'] = invoices_by_customer_type_rating.Member - invoices_by_customer_type_rating.Normal
invoices_by_customer_type_rating[invoices_by_customer_type_rating['Member Normal rating'] < -0.3]

Customer type,Branch,Product line,Member,Normal,Member Normal rating
1,A,Fashion accessories,6.6,7.1,-0.5
2,A,Food and beverages,7.0,7.5,-0.5
7,B,Fashion accessories,6.5,7.0,-0.5
11,B,Sports and travel,6.2,6.8,-0.6
12,C,Electronic accessories,6.4,6.9,-0.5


**Висновок** For the above Branches and Products lines Member customers are satisfied less than Normal customers. It makes sense to check this.

4-6. **Гіпотеза 2**

In [729]:
inv_by_cat_branch_price = pd.pivot_table(df, index=['Product line','Branch'], columns='Customer type', values='Unit price',
                                                       aggfunc='mean').round(2).reset_index()
#inv_by_cat_branch_price

In [731]:
inv_by_cat_branch_quantity = pd.pivot_table(df, index=['Product line','Branch'], columns='Customer type', values='Quantity',
                                                       aggfunc='sum').round(2).reset_index()
#inv_by_cat_branch_quantity

In [801]:
inv_by_cat_branch_income = pd.pivot_table(df, index=['Product line','Branch'], columns='Customer type', values='gross income',
                                                       aggfunc='sum').round(2).reset_index()
#inv_by_cat_branch_income

In [833]:
inv_by_cat = inv_by_cat_branch_price.join(inv_by_cat_branch_quantity, lsuffix='_price', rsuffix='_quantity').drop(
    ['Product line_quantity','Branch_quantity'],axis=1)

inv_by_cat_total = pd.concat([inv_by_cat, inv_by_cat_branch_income], axis=1).drop(['Product line', 'Branch'], axis=1).rename(
    {
        'Product line_price': 'Product_line',
        'Branch_price': 'Branch',
        'Member': 'Member_income',
        'Normal': 'Normal_income'
    }, axis=1)

inv_by_cat_total['Member_Normal_dif'] = inv_by_cat_total.Member_income - inv_by_cat_total.Normal_income
inv_by_cat_total[inv_by_cat_total.Member_Normal_dif < -50]

Customer type,Product_line,Branch,Member_price,Normal_price,Member_quantity,Normal_quantity,Member_income,Normal_income,Member_Normal_dif
1,Electronic accessories,B,48.47,51.19,147,169,353.56,458.41,-104.85
2,Electronic accessories,C,64.73,50.71,120,213,377.52,525.76,-148.24
3,Fashion accessories,A,48.43,62.92,133,130,330.60,447.14,-116.54
4,Fashion accessories,B,52.18,57.69,143,154,349.50,432.08,-82.58
13,Home and lifestyle,B,53.84,56.72,137,158,369.97,465.71,-95.74


**Висновок** For the above Branches and Products lines the Gross income from Member customers purchases are much less than from Normal customers. It makes sense to check this.
The intresting point that Electronic accessories in 'C', same Fashion accessories in 'A' and 'B' are the same combination the first and the second hypothethis. It could be that lower 'Gross income' from Customer type Member could be caused by the Member policy for those Product lines in those Branches.

In [835]:
#inv_by_cat_branch_price.groupby('Branch')['gross income'].agg('sum')
#inv_by_cat_branch_price.groupby(['Product line','Customer type','Branch'])[['Quantity']].max()
#inv_by_cat_branch_price.loc[('Electronic accessories'),(('Unit price','Quantity','gross income'),'C')].round(2)

4-6. **Гіпотеза 3**

In [943]:
def get_hour(time_value):
    return int(time_value.split(':')[0])

df['Hour'] = df['Time'].apply(get_hour)

inv_by_time = pd.pivot_table(df, index='Hour', columns='Branch', values='gross income',
                                                       aggfunc='sum').reset_index().fillna(0)
inv_by_time

Branch,Hour,A,B,C
0,10,533.7340,422.1830,540.3440
1,11,540.4710,499.1340,406.9345
2,12,451.6700,403.5910,385.9715
3,13,497.3215,536.7815,619.3840
4,14,421.5435,556.8840,489.5915
5,15,536.8430,487.6745,460.2210
6,16,517.6065,196.3600,487.2870
7,17,430.6540,373.3835,360.0205
8,18,354.6200,455.0135,429.9065
9,19,491.9170,774.4025,624.1335


In [973]:
the_most_act_hour_A = inv_by_time[inv_by_time.A > 500][['Hour', 'A']]
the_most_act_hour_A

Branch,Hour,A
0,10,533.7340
1,11,540.4710
5,15,536.8430
6,16,517.6065


In [981]:
the_most_act_hour_B = inv_by_time[inv_by_time.B > 500][['Hour', 'B']]
the_most_act_hour_B

Branch,Hour,B
3,13,536.7815
4,14,556.8840
9,19,774.4025


In [979]:
the_most_act_hour_C = inv_by_time[inv_by_time.C > 500][['Hour', 'C']]
the_most_act_hour_C

Branch,Hour,C
0,10,540.3440
3,13,619.3840
9,19,624.1335


**Висновок**
Branch A is active in the mornings and afternoons.
Branch B is active in the afternoon and evening.
Branch C is active in the morning, afternoon and evening.

On the basis of the results, we can plan the promotional activities or the working hours.